---
title: Shuttleworth-Wallace TSEB in advective conditions
subject: Tutorial
subtitle: Notebook to evaluate the new TSEB version initialized by the Shuttleworth & Wallace energy combination model
short_title: TSEB-SW
authors:
  - name: Héctor Nieto
    affiliations:
      - Instituto de Ciencias Agrarias, ICA
      - CSIC
    orcid: 0000-0003-4250-6424
    email: hector.nieto@ica.csic.es
  - name: Benjamin Mary
    affiliations:
      - Insituto de Ciencias Agrarias
      - CSIC
    orcid: 0000-0001-7199-2885
license: CC-BY-SA-4.0
keywords: TSEB, radiation, Beer-Lambert law, albedo
---

# Summary
This interactive Jupyter Notebook has the objective of showing the implemenation of TSEB-SW under advective conditions. It is based on [](https://doi.org/10.1007/s00271-022-00778-y) and [](https://doi.org/10.1007/s00271-022-00790-2).

# Instructions
Read carefully all the text and follow the instructions.

Once each section is read, run the jupyter code cell underneath (marked as `In []`) by clicking the icon `Run`, or pressing the keys SHIFT+ENTER of your keyboard. A graphical interface will then display, which allows you to interact with and perform the assigned tasks.

To start, please run the following cell to import all the packages required for this notebook. Once you run the cell below, an acknowledgement message, stating all libraries were correctly imported, should be printed on screen.

In [ ]:
%matplotlib inline
from pathlib import Path
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display
from functions import radiation_and_available_energy as fn
import numpy as np

# Shuttleworth-Wallace model

The two-source Shuttleworth-Wallace energy combination model {cite:p}`https://doi.org/10.1002/qj.49711146910` was specifically designed to account for evapotranspiration partitioning in sparse crops. Therefore, both the heat and water fluxes are separated into a soil and canopy layer, with a series of resistances set in series).
    
:::{figure} ./input/figures/shuttleworth_wallace_model.png
:alt:Shuttleworth & Wallace energy combination model
:name:shuttleworth-wallace-model
Two-Source energy balance scheme including the transport of both heat (H) and water vapour ($\lambda E$). Adapted from [](https://doi.org/10.1002/qj.49711146910).
:::

Energy fluxes are therefore split into soil and canopy, considering the conservation of energy:
    
:::{math}
  R_{n} & \approx H + \lambda E + G\\
  R_{n,S} & \approx H_{S} + \lambda E_{S} + G\\
  R_{n,C} & \approx H_{C} + \lambda E_{C}\label{eq:Energy_Balance_TSEB}
:::

with $R_n$ being the net radiation, $H$ the sensible heat flux, $\lambda E$ the latent heat flux or evapotranspiration, and $G$ the soil heat flux (all fluxes are expressed in W m$^{-2}$. The approximation in Eq. \ref{eq:Energy_Balance} reflects additional components of the energy balance that are usually neglected, such as heat advection, storage of energy in the canopy layer or energy for the fixation of CO$_2$ \citep{Hillel1998, Baldocchi1991}, which are not computed by the model.
    
Canopy latent heat flux (or transpiration) is computed as
     
:::{math}
 \lambda E_C = \frac{\Delta R_{n.C} + \rho_a c_p \frac{VPD_0}{R_x}}{\Delta + \gamma  \left(1 + \frac{R_c}{R_x}\right)}
:::

where $\rho_a$ is the air density; $c_p$ the heat capacity of air, $VPD_0$ is the air vapour pressure deficit at the canopy-air interface; $R_x$ is the canopy boundary resistance to momentum, heat and vapour transport; and $R_c$ is related to the leaf stomatal conductance $g_{s}$ as we will see later in section [](#gs-header)
    
The vapor pressure deficit at the canopy-air interface ($VPD_0$) is computed as
     
:::{math}
 VPD_0 = VPD + R_a \frac{\Delta \left(R_n - G\right) - \left(\Delta + \gamma\right) \lambda E} {\rho_a c_p}
:::

where $VPD$ is the measured atmospheric vapour pressure deficit, $R_a$ is the aerodynamic resistance to turbulent transport, $R_n$ is the surface net radiation, $G$ is the soil heat flux, and $\lambda E$ is the surface bulk (soil + canopy) latent heat flux, estimated as:
     
:::{math}
 \lambda E = C_c PM_C + C_s PM_S \label{eq:sw_weight}
:::

$PM_C$ and $PM_S$ are the estimates of an infinite deep canopy and bare soil latent heat fluxes, respectively using the Penman-Monteith equation:

:::{math}
  PM_C & = \frac{\Delta  \left(R_n - G\right) + \frac{\rho_a c_p VPD - \Delta  R_x \left(R_{n,S} - G\right)}{
      R_a + R_x}}{\Delta + \gamma \left(1 + \frac{R_c}{R_a + R_x}\right)}\\
  PM_S & = \frac{\Delta  \left(R_n - G\right) + \frac{\rho_a c_p VPD - \Delta R_s R_{n,C}}{R_a + R_s}}{\Delta + \gamma \left(1 + \frac{R_{ss}} {R_a + R_s}\right)}
::::
    
where $R_s$ is the soil boundary layer resistance to turbulent transport and $R_{ss}$ is the near-surface soil resistance to vapour transport. The latter is set to a fixed value of $R_{ss}=2000$ s m$^{-1}$ considering a rather dry soil surface, in order to be consistent with the definition of potential ET adopted with the Penman-Monteith approach.

:::{seealso}
:class:dropdown
The full code for the Shuttleworth and Wallace model is at the [pyTSEB GitHub repository](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/energy_combination_ET.py#L277)
:::

(gs-header)=
## Sensitivity of stomatal conductance to vapour pressure deficit

[](https://doi.org/10.1007/s00271-022-00778-y) showed the advantages of accounting for the sensitivity of stomatal closure at higher VPD in canopies highly coupled with the atmosphere {cite:p}`https://doi.org/10.1016/S0065-2504(08)60119-1`. 

We are going to evaluate the sensitivity of leaf stomata to VPD using estimates of transpiration derived from the Eddy Covariance. This approach is based on the method proposed by [](https://doi.org/10.1111/j.1365-3040.1995.tb00371.x) 
    
The canopy stomatal resistance ($R_c$), dependent of VPD variations, is then computed as. 
:::{math}
:label: R_c
R_c = \frac{1}{g_{s,0} f_s f_g LAI}
:::

where $LAI$ is the leaf area index, defined as half of the total leaf area, and $f_s$ is a factor representing the distribution of stomata in the leaf ($f_s=1$ for hypostomatous leaves and $f_s=2$ for amphistomatous leaves) and $f_g$ is the fraction of LAI that is green and hence actively transpiring.

Accounting for the negative feedback observed between transpiration (T) rates and stomatal closure based on a wide variety of plant level measurements, [](https://doi.org/10.1111/j.1365-3040.1995.tb00371.x) proposed a method to parameterize the relationship between leaf stomatal conductance ($g_s$) and $VPD$, based on measurements of transpiration as follows:

:::{math}
:label: monteith1995
g_{s} = \frac{g_m}{1 + g_m VPD/T_m}
:::

Values of $g_m$ and $T_m$ will be empirically derived by plotting observations of $T$ and $VPD$, and building a linear regression:

:::{math}
:label: vpd-linear
1/T = \frac{1}{a{VPD}} + b
:::

from which $g_m = a$ and $T_m = 1/b$. Finally, from these metrics qe can derive the [](https://doi.org/10.1111/j.1365-3040.1995.tb00370.x) stomatal conductance model:
    
:::{math}
:label: leuning1995
g_{s,0} = \frac{g_m}{1 + VPD / D_0}
:::

### Application with actual data
To build the linear regression of Eq. [](#vpd-linear) we need work at leaf scale. Therefore, the estimated canopy transpiration from the eddy covariance tower is (TEC) first downscaled to an effective leaf transpiration rate using the satellite LAI ($T_{leaf} = T/LAI$). 

First select the sites that you want to use to derived the $g_m$, $T_m$ and $D_0$ metrics. You can select one or several sites and see how sensitive are these parameters to the grape varieties.

In [ ]:
w_site_list = widgets.SelectMultiple(
    options=[('Sierra Loma N', "slmN"), ('Sierra Loma S', "slmS"), 
             ('Barrelli 007', "bar007"), ('Barrelli 012', "bar012"),
             ('Ripperdan 760', "rip760"), 
             ('Ripperdan 720-1', "rip720_1"),  ('Ripperdan 720-2', "rip720_2"),  ('Ripperdan 720-3', "rip720_3"),  ('Ripperdan 720-4', "rip720_4")],
    value=["rip720_1", "rip720_2", "rip760"],
    description='Sites',
    rows=9
)
display(w_site_list)

#### Read the EC and LAI data files

In [ ]:
# Import Python libraries
from pathlib import Path
import pandas as pd
from pyTSEB import TSEB

# Set the LAI and readiation folders
input_dir = Path().absolute() / "input"
lai_dir = input_dir / "canopy"
rad_dir = input_dir / "meteo"

# Append all tables in a single merged table
ec_all = pd.DataFrame()
for site in w_site_list.value:
    # Set the input files based on the chosen site
    lai_filename = lai_dir / f"FLX_US-{site}_FLUXNET2015_AUXCANOPY_DD.csv"
    ec_filename = rad_dir / f"FLX_US-{site}_FLUXNET2015_SUBSET_HR.csv"
    print(f"LAI file path is {lai_filename}")
    print(f"Micrometeorogy file path is {ec_filename}")
    
    # Read the LAI and radiation tables
    lai = pd.read_csv(lai_filename, sep=";", na_values=-9999)
    ec = pd.read_csv(ec_filename, sep=";", na_values=-9999)
    
    # Merge both tables by date
    ec["TIMESTAMP"] = pd.to_datetime(ec["TIMESTAMP"], format="%Y%m%d%H%M")
    lai["DATE"] = pd.to_datetime(lai["TIMESTAMP"], format="%Y%m%d").dt.date
    lai = lai.drop(labels=["TIMESTAMP"], axis=1)
    ec["DATE"] = ec["TIMESTAMP"].dt.date
    ec = ec.merge(lai, on="DATE")
    ec["SITE"] = site
    

    ec_all = pd.concat([ec_all, ec], ignore_index=True, axis=0)

::::{seealso}
:class:dropdown
This is a full description of the EC column fields:

[](./fluxnet2015_variables.md)
::::

The values of $T$ are estimated using a several methods based on relaxed-eddy accumulation and quadrant analysis {cite:p}`https://doi.org/10.1016%2Fj.agrformet.2008.03.002;https://doi.org/10.1016/j.agrformet.2021.108790`. We compute the average of the three methods (CEC, REA, and FVS). Then we convert those values expressed in energy units into mole fractions:

In [ ]:
from pyTSEB import physiology as lf

# Only derive the parameters when grapevine in active
DOY_LIMS = (120, 246)

ec_all["LE_C"] = np.nanmean(ec_all[["LE_C_ECC", "LE_C_REA", "LE_C_FVS"]], axis=1)
# Convert Celsius to Kelvin
ec_all["TA"] = ec_all["TA"] + 273.15
# Convert kPa to hPa
ec_all["PA"] = 10 * ec_all["PA"]

# Work only under daytime conditions and positive LE fluxes
valid = np.logical_and.reduce((ec_all["SW_IN"] > 100,
                               ec_all["VPD"] > 0,
                               np.isfinite(ec_all["LE_C"]),
                               ec_all["TIMESTAMP"].dt.dayofyear >= DOY_LIMS[0],
                               ec_all["TIMESTAMP"].dt.dayofyear <= DOY_LIMS[1],
                               ec_all["LE_C"] > 0))

ec_all = ec_all.loc[valid, :]
# Monteith approach needs to convert units to millimoles
ec_all["VPD_MMOL"] = 1e3 * ec_all["VPD"].values / ec_all["PA"].values
ec_all["LE_C_MMOL"] = ec_all["LE_C"].values / (ec_all["LAI"].values * lf.mmolh20_2_wm2(ec_all["TA"].values))

The linear relationship between the reciprocals of VPD and maximum T only appears when VPD is the main limiting factor to stomata closure {cite:p}`https://doi.org/10.1111/j.1365-3040.1995.tb00371.x`. Therefore, this linear relationship is derived from the lower envelope of the 1/T vs. 1/VPD scatterplot for all daytime EC observations under relatively high VPD values.

We create the following helper functions to get the lower envelope of the vpd vs. T relationship and the stomatal metrics:

In [ ]:
def get_envelope(x_array, y_array, percentile=0.95, n_bins=30):
    # Bin x_array
    _, bin_edges = np.histogram(x_array, bins=n_bins)

    x_values = []
    y_values = []
    for i, edge in enumerate(bin_edges[:-1]):
        valid = np.logical_and(x_array >= edge, x_array < bin_edges[i + 1])
        if np.any(valid):
            x_values.append(np.mean(x_array[valid]))
            if percentile == "minimum":
                y_values.append(np.nanmin(y_array[valid]))
            elif percentile == "maximum":
                y_values.append(np.nanmax(y_array[valid]))
            else:
                y_values.append(np.percentile(y_array[valid], 100 * percentile))

    return np.asarray(x_values), np.asarray(y_values)

    
def monteith_2_leuning(g_m, t_m, p=1013.15):
    """

    :param g_m: mol m$^{{-2}}$ s$^{{-1}}$
    :param t_m: mmol m$^{{-2}}$ s$^{{-1}}$
    :return:
    d_0: hPa
    """
    d_0 = t_m / g_m  # mmol mol-1
    # Convert from mmol mol-1 to mb
    d_0 *= 1e-3 * p
    return d_0


def leuning_stress(vpd, d_0):

    stress = 1. / (1. + vpd / d_0)
    return stress


... and with these helper functions we are ready to estimate the stomatal parameters

In [ ]:
import scipy.stats as st
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Set the criteria for the envelope search
MIN_PERCENTILE = 0.01
N_BINS = "auto"

# We first dervive the parameters based on all the selected sites together
valid = 1. / ec_all["VPD_MMOL"] < 0.08

print(f"Finding stomatal parameters for all sites together")
# Find the lower envelope                         
xs, ys = get_envelope(1. / ec_all.loc[valid, "VPD_MMOL"], 1. / ec_all.loc[valid, "LE_C_MMOL"],
                      percentile=MIN_PERCENTILE,
                      n_bins=N_BINS)

# Adjust a linear model from the lower envelope
lm = st.linregress(xs, ys)
# Plot the linear model
fig = go.Figure()
fig.add_trace(go.Scatter(x=xs, y=lm.intercept + lm.slope * xs, name="Fit ALL", mode="lines", line={"color":"black"}))

# We will store each parameter in a Python dictionary
n_cases = {}
gm = {}
t_m = {}
d_0 = {}

# Derive the Monteith and Leuning stomata parameters
n_cases["ALL"] = np.sum(valid)
gm["ALL"] = 1. / lm.slope
t_m["ALL"] = 1. / lm.intercept
d_0["ALL"] = monteith_2_leuning(gm["ALL"], t_m["ALL"], p=np.nanmean(ec_all.loc[valid, "PA"]))

# We now repeat the same procedure for each site individually
for site, site_label in zip(w_site_list.value, w_site_list.label):
    print(f"Finding stomatal parameters for {site_label}")
    valid = np.logical_and(1. / ec_all["VPD_MMOL"] < 0.08, ec_all["SITE"] == site)
    
    # Find the lower envelope                         
    xs, ys = get_envelope(1. / ec_all.loc[valid, "VPD_MMOL"], 1. / ec_all.loc[valid, "LE_C_MMOL"],
                          percentile=MIN_PERCENTILE,
                          n_bins=N_BINS)
    # Adjust a linear model from the lower envelope
    lm = st.linregress(xs, ys)
    # Plot the linear model
    fig.add_trace(go.Scatter(x=xs, y=lm.intercept + lm.slope * xs, name=f"Fit {site}", mode="lines")) 

    # Derive the Monteith and Leuning stomata parameters
    n_cases[site] = np.sum(valid)
    gm[site] = 1. / lm.slope
    t_m[site] = 1. / lm.intercept
    d_0[site] = monteith_2_leuning(gm[site], t_m[site], p=np.nanmean(ec_all.loc[valid, "PA"]))
    fig.add_trace(go.Scatter(x=1. / ec_all.loc[valid, "VPD_MMOL"], y=1. / ec_all.loc[valid, "LE_C_MMOL"], name=site, mode="markers"))  

fig.update_layout(title_text=f"Derivation of Stomata parameters", 
                  xaxis_title="1/VPD (mol mmol-1)", yaxis_title="1/T (s m2 mmol-1)",
                  yaxis_range=[0, 0.5], xaxis_range=[0.01, 0.1])



In [ ]:
from tabulate import tabulate
table = [#["N cases"] + [value for key, value in n_cases.items()],
         ["$g_m$ mol m$^{{-2}}$ s$^{{-1}}$"] + [value for key, value in gm.items()],
         ["$T_m$ mmol m$^{{-2}}$ s$^{{-1}}"] + [value for key, value in t_m.items()],
         ["$D_0$ hPa"] + [value for key, value in d_0.items()]]

print(tabulate(table, headers=["Parameter"] + [i for i in gm.keys()]))

# Implementation with actual data in [pyTSEB](https://github.com/hectornieto/pyTSEB)

As with the previous exercices, we will work with any of the sites of GRAPEX, using the micrometeorological measurements and Earth Observation LAI.

:::{table}
:name: tab-site-description
Description of GRAPEX sites

Site | Latitude | Longitude | Elevation (m) | Row direction (deg.) | Row spacing (m) | Min. height (m) | Max. height (m) | Min. width (m) | Max. width (m) | TA height (m) | WS height (m)
:-- | --: | --: | --: | --: | --: | --: | --: | --: | --: | --: | --:
Ripperdan 720-1 | 36.849 | -120.176 | 61 | 90 | 3.35 | 1.2 | 2.2 | 0.5 | 2.25 | 4 | 4
Ripperdan 720-2 | 36.849 | -120.174 | 61 | 90 | 3.35 | 1.2 | 2.2 | 0.5 | 2.25 | 4 | 4
Ripperdan 720-3 | 36.848 | -120.176 | 61 | 90 | 3.35 | 1.2 | 2.2 | 0.5 | 2.25 | 4 | 4
Ripperdan 720-4 | 36.848 | -120.174 | 61 | 90 | 3.35 | 1.2 | 2.2 | 0.5 | 2.25 | 4 | 4
Barrelli_007 | 38.753 | -122.98 | 113 | 135 | 3.35 | 1.25 | 2.3 | 0.5 | 1.8 | 4 | 4
Barrelli_012 | 38.751369 | -122.974658 | 112 | 45 | 3.35 | 1.25 | 2.3 | 0.5 | 1.8 | 4.4 | 4.4
Sierra Loma-north | 38.289338 | -121.117764 | 38 | 90 | 3.35 | 1.42 | 2.25 | 0.5 | 2.6 | 5 | 5
Sierra Loma-south | 38.280488 | -121.117597 | 37 | 90 | 3.35 | 1-42 | 2.25 | 0.5 | 2 | 5 | 5
Ripperdan 760 | 36.839025 | -120.21014 | 58 | 90 | 2.74 | 1.2 | 2.5 | 0.5 | 1.8 | 5.5 | 5
Barrelli_016 | 38.747 | -122.963 | 112 | 135 | 3.35 | 1.25 | 2.3 | 0.5 | 1.8 | 4 | 4
:::

## Select a site

In [ ]:
w_site = widgets.Dropdown(
    options=[('Sierra Loma N', "slmN"), ('Sierra Loma S', "slmS"), 
             ('Barrelli 007', "bar007"), ('Barrelli 012', "bar012"),
             ('Ripperdan 760', "rip760"), 
             ('Ripperdan 720-1', "rip720_1"),  ('Ripperdan 720-2', "rip720_2"),  ('Ripperdan 720-3', "rip720_3"),  ('Ripperdan 720-4', "rip720_4")],
    value="rip760",
    description='Site:',
)
display(w_site)

## Read the LAI and Micrometeorology data

In [ ]:
# Import Python libraries
from pathlib import Path
import pandas as pd

# Set the LAI and readiation folders
input_dir = Path().absolute() / "input"
lai_dir = input_dir / "canopy"
ec_dir = input_dir / "meteo"
# Set the input files based on the chosen site
lai_filename = lai_dir / f"FLX_US-{w_site.value}_FLUXNET2015_AUXCANOPY_DD.csv"
ec_filename = ec_dir / f"FLX_US-{w_site.value}_FLUXNET2015_SUBSET_HR.csv"
print(f"LAI file path is {lai_filename}")
print(f"EC file path is {ec_filename}")

# Read the LAI and radiation tables
lai = pd.read_csv(lai_filename, sep=";", na_values=-9999)
ec = pd.read_csv(ec_filename, sep=";", na_values=-9999)

# Merge both tables by date
ec["TIMESTAMP"] = pd.to_datetime(ec["TIMESTAMP"], format="%Y%m%d%H%M")
lai["DATE"] = pd.to_datetime(lai["TIMESTAMP"], format="%Y%m%d").dt.date
lai = lai.drop(labels=["TIMESTAMP"], axis=1)
ec["DATE"] = ec["TIMESTAMP"].dt.date
ec = ec.merge(lai, on="DATE")

# We discard all cases at night (SW_IN <=0)
ec = ec[ec["SW_IN"] > 0]

# Convert Celsius to Kelvin
ec["TA"] = ec["TA"] + 273.15
# Convert kPa to hPa
ec["PA"] = 10 * ec["PA"]

# Evaluate different energy balance closure corrections
ec["H_RES"] = ec["NETRAD"] - ec["G"] - ec["LE"]
ec['LE_RES'] = ec['NETRAD'] - ec['G'] - ec['H']
ec["LE_BR"], ec["H_BR"] = TSEB.met.bowen_ratio_closure(ec["NETRAD"], ec["G"],
                                                       ec["H"], ec["LE"])
# Mean of uncorrected, ressidual and Bowen Ratio
ec["H_ENS"] = np.nanmean([ec["H_RES"], ec["H_BR"], ec['H']], axis=0)
ec["LE_ENS"] = np.nanmean([ec["LE_RES"], ec["LE_BR"], ec['LE']], axis=0)

## Set the stomata sensitivty to VPD

In [ ]:
style = {'description_width': 'initial'}
w_d_0 = widgets.BoundedFloatText(min=0.01, max=20, value=16.93, description="$D_0$ (hPa)", step=0.1, style=style)
w_g_m = widgets.BoundedFloatText(min=0.01, max=1, value=0.53, description="$g_m$ (mol m$^{-2}$ s$^{-1}$)", step=0.01, style=style)
display(w_d_0, w_g_m)

## Get the canopy a priori structural properties

In [ ]:
import yaml
yaml_file = input_dir / "site_description.yaml"
site_dict = yaml.safe_load(yaml_file.read_text())
lat = float(site_dict["lat"][w_site.value])
lon = float(site_dict["lon"][w_site.value])
elev = float(site_dict["elev"][w_site.value])
row_direction = float(site_dict["row_direction"][w_site.value])
interrow = float(site_dict["interrow"][w_site.value])
hc_min = float(site_dict["hc_min"][w_site.value])
hc_max = float(site_dict["hc_max"][w_site.value])
wc_min = float(site_dict["wc_min"][w_site.value])
wc_max = float(site_dict["wc_max"][w_site.value])
zh = float(site_dict["zh"][w_site.value])
zm = float(site_dict["zm"][w_site.value])

print(f"{w_site.label} has the following site characteristics:\n"
      f"\t Latitude: {lat} deg. \n"
      f"\t Longitude: {lon} deg. \n"
      f"\t Elevation: {elev} m \n"
      f"\t Row direction: {row_direction} deg. \n"
      f"\t Row spacing: {interrow} m \n"
      f"\t Min. canopy height: {hc_min} m \n"
      f"\t Max. canopy height: {hc_max} m \n"
      f"\t Min. canopy width: {wc_min} m \n"
      f"\t Max. canopy width: {wc_max} m \n"
      f"\t Air temperature measurement height: {zh} m \n"
      f"\t Wind speed temperature measurement height: {zh} m"
)

## Estimate the structural variables based on Earth Observation LAI timeseries
As with the previous exercise, we will estimate timeseries of canopy structural variables based on LAI and empirical observations of trellis development.

:::{figure} ./input/figures/vineyard_structural_functions.png
:alt: Empirical structural functions
:name: structural-functions
Empirical models relating canopy height , canopy width and the bottom of the canopy with the fused STARFM LAI. Solid dots represent in situ measured values
:::
These empirical equations are coded in Python as:

In [ ]:
import numpy as np

################################################################################
# Empirical equations to generate canopy structural parameters from LAI
################################################################################
def lai_2_hc(lai, hc_min):
    slope = 0.42
    hc = hc_min + slope * lai
    return hc


def lai_2_hbratio(lai, hc_min):
    hb_ratio_mean = 0.4848388065
    hb_ratio = np.zeros(lai.shape)

    hb_ratio[lai < hc_min] = 1. + ((hb_ratio_mean - 1.) / hc_min) * lai[
        lai < hc_min]
    hb_ratio[lai >= hc_min] = hb_ratio_mean

    return hb_ratio


def lai_2_width(lai, wc_min, wc_max):
    beta = 6.96560056
    offset = 1.70825736

    width = wc_min + (wc_max - wc_min) / (1.0 + np.exp(-beta * (lai - offset)))

    return width


def lai_2_fcover(lai, fc_min, fc_max):
    beta = 7.0
    offset = 1.70

    fcover = fc_min + (fc_max - fc_min) / (1.0 + np.exp(-beta * (lai - offset)))

    return fcover


def lai_2_canopy(lai, hc_min, fc_min, fc_max):
    hc = lai_2_hc(lai, hc_min)
    hb_ratio = lai_2_hbratio(lai, hc_min)
    fcover = lai_2_fcover(lai, fc_min, fc_max)

    return hc, hb_ratio, fcover

From these empirical functions together with the site description properties described in [](#site-description) we can caluculate the time series of canopy structural variables needed to compute the shadowing as shown in [](#row-crop-model)

:::{note}
:class: dropdown
These equations are site specific and most likely not applicable to other sites and crops. For operational purposes, such as using Earth Observation/satellite data a trade-off must be made between accuracy and applicability.
:::

In [ ]:
# Compute the expected minimum and maximum canopy cover based on mininum and maximum canopy width
fc_min = wc_min / interrow
fc_max = wc_max / interrow
h_c, hb_ratio, f_c = lai_2_canopy(ec["LAI"].values,
                                  hc_min,
                                  fc_min,
                                  fc_max)

# Ensure that both canopy height and cover are within the limits
h_c = np.clip(h_c, hc_min, hc_max)
f_c = np.clip(f_c, 0, 1)

## We can get the shortwave net radiation for row crops

In [ ]:
from pyTSEB import TSEB

# We can get the leaf and soil spectral from the values above, or hard code the corresponding values
rho_leaf_vis = 0.054
rho_leaf_nir = 0.262
tau_leaf_vis = 0.038
tau_leaf_nir = 0.333
rho_soil_vis = 0.07
rho_soil_nir = 0.32

# canopy width
w_c = f_c * interrow
# Canopy width to height ratio
w_c_ratio = w_c / (h_c - hb_ratio * h_c)
# Local LAI
F = ec["LAI"].values / f_c

# The time zone is PST, which corresponds to -120 deg, time longitude
stdlon = -120
# Call calc_sun_angles based on site coordinates and timestamp
sza, saa = TSEB.met.calc_sun_angles(
    np.full_like(ec['LAI'].values, lat),
    np.full_like(ec["LAI"].values, lon),
    np.full_like(ec["LAI"].values, stdlon),
    ec['TIMESTAMP'].dt.dayofyear.values,
    ec['TIMESTAMP'].dt.hour.values + ec['TIMESTAMP'].dt.minute.values / 60.)

# Compute the relative sun-row azimuth angle
psi = row_direction - saa
# Compute the clumping index for row crops
omega = TSEB.CI.calc_omega_rows(ec["LAI"], f_c, theta=sza,
                                psi=psi, w_c=w_c_ratio)

# And the effective LAI is the product of local LAI and the clumping index
lai_eff = F * omega

# Estimates the direct and diffuse solar radiation
difvis, difnir, fvis, fnir = TSEB.rad.calc_difuse_ratio(ec["SW_IN"].values,
                                                        sza,
                                                        press=np.full_like(sza, 1013.15))
par_dir = fvis * (1. - difvis) * ec["SW_IN"].values
nir_dir = fnir * (1. - difnir) * ec["SW_IN"].values
par_dif = fvis * difvis * ec["SW_IN"].values
nir_dif = fnir * difnir * ec["SW_IN"].values

# Compute the canopy and soil net radiation using Cambpell RTM
sn_c, sn_s = TSEB.rad.calc_Sn_Campbell(ec["LAI"].values,
                                       sza,
                                       par_dir + nir_dir,
                                       par_dif + nir_dif,
                                       fvis,
                                       fnir,
                                       np.full_like(sza, rho_leaf_vis),
                                       np.full_like(sza, tau_leaf_vis),
                                       np.full_like(sza, rho_leaf_nir),
                                       np.full_like(sza, tau_leaf_nir),
                                       np.full_like(sza, rho_soil_vis),
                                       np.full_like(sza, rho_soil_nir),
                                       x_LAD=1,
                                       LAI_eff=lai_eff)

sn_c[~np.isfinite(sn_c)] = 0
sn_s[~np.isfinite(sn_s)] = 0

### Evaluate the shortwave net radiation estimates

In [ ]:
from model_evaluation import double_collocation as dc
daytime = ec["SW_IN"] > 100

sn_model = sn_c + sn_s
sn_obs = ec["SW_IN"].values - ec["SW_OUT"].values

fig = go.Figure()
fig.add_trace(go.Scattergl(x=sn_model[daytime], y=sn_obs[daytime], 
                         name="Sn for row crops", mode="markers"))
fig.add_trace(go.Scatter(x=[0, 1000], y=[0, 1000], mode="lines", name="1:1 line", line={"color": "black", "dash": "dash"}))
fig.update_layout(title_text=f"Observed vs. Estimated below net radiation at {w_site.label}",
                  yaxis_range=[0, 1000], xaxis_range=[0, 1000],
                  xaxis_title="Estimated (W m-2)", yaxis_title="Observed (W m-2)")

## Set additional site parameters needed for TSEB

In [ ]:
# Grapevine leaf width
leaf_width = 0.10

# Roughness for bare soil
z0_soil = 0.15

# Kustas and Norman boundary layer resistance parameters
roil_resistance_c_param = 0.0038
roil_resistance_b_param = 0.012
roil_resistance_cprime_param = 90.
# Thermal spectra
e_v = 0.99  # Leaf emissivity
e_s = 0.94  # Soil emissivity

# Resistance for soil evaporation of a well watered situation
rss_min = 0  # Asumme initial free water evaporation

## Estimate surface aerodynamic roughness
We estimate aerodynamic roughness for tall canopies based on [](https://doi.org/10.1007/BF00709229) and [](http://dx.doi.org/10.1016/S0168-1923(00)00153-2.):

In [ ]:
z_0m, d_0 = TSEB.res.calc_roughness(ec["LAI"], h_c, w_c_ratio, np.full_like(ec["LAI"], TSEB.res.BROADLEAVED_D), f_c=f_c)
# Ensure realistic values
d_0[d_0 < 0] = 0
z_0m[np.isnan(z_0m)] = z0_soil
z_0m[z_0m < z0_soil] = z0_soil

## We derive the half-hourly LST based on pyrgeomters on the EC tower

In [ ]:
# Surface emissivity
e_surf = f_c * e_v + (1. - f_c) * e_s

# LST from longwave radiometers
lst = ((ec['LW_OUT'].values - (1. - e_surf) * ec['LW_IN'].values) / (
        TSEB.rad.SB * e_surf)) ** 0.25

# And it is assuming that the radiometer is looking at nadir
vza = np.zeros_like(lst)

## We run now TSEB-SW

:::{seealso}
:class:dropdown
The full code for the Shuttleworth and Wallace model is at the [pyTSEB GitHub repository](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/TSEB.py#L946)
:::

In [ ]:
# Initialize the potential stomtal conductance by a minimum value
gs = np.full_like(lst, 0.01)

# Update the  potential stomtal conductance for daytime
gs[daytime] = w_g_m.value * leuning_stress(ec["VPD"][daytime],  w_d_0.value)

# Convert stomtata conductance to resistance (s m-1)
rst_min = 1. / (TSEB.res.molm2s1_2_ms1(ec["TA"].values, ec["PA"].values) * gs)
rst_min[rst_min < 0] = np.nan

resistance_flag = [0, {"KN_c": np.full_like(lst, roil_resistance_c_param),
                       "KN_b": np.full_like(lst, roil_resistance_b_param),
                       "KN_C_dash": np.full_like(lst, roil_resistance_cprime_param)}]

# Run TSEB-SW
[flag_sw, ts_sw, tc_sw, t_ac_sw, ln_s_sw, ln_c_sw, le_c_sw, h_c_sw, le_s_sw, h_s_sw, g_sw,
 r_s_sw, r_x_sw, r_a_sw, rss_sw, rst_sw, u_friction_sw, lmo_sw, n_iterations_sw] = TSEB.TSEB_SW(                                                     
     lst,
     vza,
     ec["TA"].values,
     ec["WS"].values,
     ec["EA"].values,
     ec["PA"].values,
     sn_c,
     sn_s,
     ec["LW_IN"].values,
     ec["LAI"].values,
     h_c,
     e_v,
     e_s,
     z_0m,
     d_0,
     zm,
     zh,
     x_LAD=np.ones_like(lst),
     f_c=f_c,
     f_g=np.ones_like(lst),
     w_C=w_c_ratio,
     leaf_width=leaf_width,
     z0_soil=z0_soil,
     Rst_min=rst_min,
     Rss_min=rss_min,
     resistance_form=resistance_flag,
     calcG_params=[[1], 0.35])

# ... and finally we compute the bulk fluxes
le_sw = le_c_sw + le_s_sw
h_sw = h_c_sw + h_s_sw
netrad_sw = sn_c + sn_s + ln_c_sw + ln_s_sw

### ... and to compare we also run TSEB-PT

In [ ]:
# Run TSEB-PT
alpha_PT_0 = 1.26
[flag_pt, ts_pt, tc_pt, t_ac_pt, ln_s_pt, ln_c_pt, le_c_pt, h_c_pt, le_s_pt, h_s_pt, g_pt,
 r_s_pt, r_x_pt, r_a_pt, u_friction_pt, lmo_pt, n_iterations_pt] = TSEB.TSEB_PT(                                                     
     lst,
     vza,
     ec["TA"].values,
     ec["WS"].values,
     ec["EA"].values,
     ec["PA"].values,
     sn_c,
     sn_s,
     ec["LW_IN"].values,
     ec["LAI"].values,
     h_c,
     e_v,
     e_s,
     z_0m,
     d_0,
     zm,
     zh,
     x_LAD=np.ones_like(lst),
     f_c=f_c,
     f_g=np.ones_like(lst),
     w_C=w_c_ratio,
     leaf_width=leaf_width,
     z0_soil=z0_soil,
     alpha_PT=alpha_PT_0,
     resistance_form=resistance_flag,
     calcG_params=[[1], 0.35])

# ... and we compute the bulk fluxes
le_pt = le_c_pt + le_s_pt
h_pt = h_c_pt + h_s_pt
netrad_pt = sn_c + sn_s + ln_c_pt + ln_s_pt

### ... and with TSEB-PM

In [ ]:
# Run TSEB-PT
r_c_min = rst_min / ec["LAI"].values
[flag_pm, ts_pm, tc_pm, t_ac_pm, ln_s_pm, ln_c_pm, le_c_pm, h_c_pm, le_s_pm, h_s_pm, g_pm,
 r_s_pm, r_x_pm, r_a_pm, u_friction_pm, lmo_pm, n_iterations_pm] = TSEB.TSEB_PM(                                                     
     lst,
     vza,
     ec["TA"].values,
     ec["WS"].values,
     ec["EA"].values,
     ec["PA"].values,
     sn_c,
     sn_s,
     ec["LW_IN"].values,
     ec["LAI"].values,
     h_c,
     e_v,
     e_s,
     z_0m,
     d_0,
     zm,
     zh,
     x_LAD=np.ones_like(lst),
     f_c=f_c,
     f_g=np.ones_like(lst),
     w_C=w_c_ratio,
     leaf_width=leaf_width,
     z0_soil=z0_soil,
     r_c_min=r_c_min,
     resistance_form=resistance_flag,
     calcG_params=[[1], 0.35])

# ... and we compute the bulk fluxes
le_pm = le_c_pm + le_s_pm
h_pm = h_c_pm + h_s_pm
netrad_pm = sn_c + sn_s + ln_c_pm + ln_s_pm

## Finally we compare the different retrievals
Select which method of EB correction you want to apply during the validation:

In [ ]:
w_ebc = widgets.Dropdown(
    options=[('No EB correction', ("LE", "H")), 
             ('Residual to LE', ("LE_RES", "H")), 
             ('Residual to H', ("LE_RES", "H")), 
             ('Bowen Ratio', ("LE_BR", "H_BR")),
             ('Ensemble correction', ("LE_ENS", "H_ENS"))],
    value=("LE_ENS", "H_ENS"),
    description='EBC method:',
)
display(w_ebc)

In [ ]:
from model_evaluation import double_collocation as dc
from tabulate import tabulate
LE_OBS, H_OBS = w_ebc.value
print(f"Using {LE_OBS} and {H_OBS} fields for validating respectively  LE and H")

le_marker = {"color": "blue", "size": 3}
h_marker = {"color": "red", "size": 3}
g_marker = {"color": "green", "size": 3}
rn_marker = {"color": "black", "size": 3}

fig = make_subplots(rows=1, cols=3,
                    shared_yaxes=True,
                    horizontal_spacing=0.01,
                   subplot_titles=("TSEB-SW", "TSEB-PT", "TSEB-PM"))

valid_sw = np.logical_and(flag_sw < 5, daytime)
valid_pt = np.logical_and(flag_pt < 5, daytime)
valid_pm = np.logical_and(flag_pm < 5, daytime)


fig.add_trace(go.Scattergl(x=netrad_sw[valid_sw], y=ec.loc[valid_sw, "NETRAD"], 
                         name="Rn TSEB-SW", mode="markers", marker=rn_marker),
             row=1, col=1)

fig.add_trace(go.Scattergl(x=netrad_pt[valid_pt], y=ec.loc[valid_pt, "NETRAD"], 
                         name="Rn TSEB-PT", mode="markers", marker=rn_marker),
             row=1, col=2)

fig.add_trace(go.Scattergl(x=netrad_pm[valid_pm], y=ec.loc[valid_pm, "NETRAD"], 
                         name="Rn TSEB-Pm", mode="markers", marker=rn_marker),
             row=1, col=3)

fig.add_trace(go.Scattergl(x=g_sw[valid_sw], y=ec.loc[valid_sw, "G"], 
                         name="G TSEB-SW", mode="markers", marker=g_marker),
             row=1, col=1)

fig.add_trace(go.Scattergl(x=g_pt[valid_pt], y=ec.loc[valid_pt, "G"], 
                         name="G TSEB-PT", mode="markers", marker=g_marker),
             row=1, col=2)

fig.add_trace(go.Scattergl(x=g_pm[valid_pm], y=ec.loc[valid_pm, "G"], 
                         name="G TSEB-PM", mode="markers", marker=g_marker),
             row=1, col=3)

fig.add_trace(go.Scattergl(x=le_sw[valid_sw], y=ec.loc[valid_sw, LE_OBS], 
                         name="LE TSEB-SW", mode="markers", marker=le_marker),
             row=1, col=1)

fig.add_trace(go.Scattergl(x=le_pt[valid_pt], y=ec.loc[valid_pt, LE_OBS], 
                         name="LE TSEB-PT", mode="markers", marker=le_marker),
             row=1, col=2)

fig.add_trace(go.Scattergl(x=le_pm[valid_pm], y=ec.loc[valid_pm, LE_OBS], 
                         name="LE TSEB-Pm", mode="markers", marker=le_marker),
             row=1, col=3)

fig.add_trace(go.Scattergl(x=h_sw[valid_sw], y=ec.loc[valid_sw, H_OBS], 
                         name="H TSEB-SW", mode="markers", marker=h_marker),
             row=1, col=1)

fig.add_trace(go.Scattergl(x=h_pt[valid_pt], y=ec.loc[valid_pt, H_OBS], 
                         name="H TSEB-PT", mode="markers", marker=h_marker),
             row=1, col=2)

fig.add_trace(go.Scattergl(x=h_pm[valid_pm], y=ec.loc[valid_pm, H_OBS], 
                         name="H TSEB-PM", mode="markers", marker=h_marker),
             row=1, col=3)

fig.add_trace(go.Scatter(x=[-200, 800], y=[-200, 800], mode="lines", name="1:1 line", line={"color": "black", "dash": "dash"}),
             row=1, col=1)
fig.add_trace(go.Scatter(x=[-200, 800], y=[-200, 800], mode="lines", name="1:1 line", line={"color": "black", "dash": "dash"}),
             row=1, col=2)
fig.add_trace(go.Scatter(x=[-200, 800], y=[-200, 800], mode="lines", name="1:1 line", line={"color": "black", "dash": "dash"}),
             row=1, col=3)


fig.update_layout(title_text=f"Observed vs. Estimated fluxes at {w_site.label}",
                  yaxis_range=[-200, 800], xaxis_range=[-200, 800], 
                  xaxis_title="Estimated (W m-2)", yaxis_title="Observed (W m-2)")


### Evaluate the errors in H and LE
In this last step you can evaluate the error metric for both TSEB-SW and TSEB-PT models. In addition, you can filter by observed values of sensible heat flux to see better the performance of both models under advective condition, such as negative H at daytime.

In [ ]:
w_h_lims = widgets.IntRangeSlider(value=[-200, 800], min=-200, max=800, step=1, description='Validate on a range for H:', 
                                  style={'description_width': 'auto'}, layout=widgets.Layout(width='50%'))
display(w_h_lims)

In [ ]:

valid_sw_2 = np.logical_and.reduce((valid_sw, ec[H_OBS] >= w_h_lims.value[0], ec[H_OBS] <= w_h_lims.value[1]))
valid_pt_2 = np.logical_and.reduce((valid_pt, ec[H_OBS] >= w_h_lims.value[0], ec[H_OBS] <= w_h_lims.value[1]))
valid_pm_2 = np.logical_and.reduce((valid_pm, ec[H_OBS] >= w_h_lims.value[0], ec[H_OBS] <= w_h_lims.value[1]))


h_sw_bias, h_sw_mae, h_sw_rmse = dc.error_metrics(ec.loc[valid_sw_2, H_OBS].values, h_sw[valid_sw_2])
h_pt_bias, h_pt_mae, h_pt_rmse = dc.error_metrics(ec.loc[valid_pt_2, H_OBS].values, h_pt[valid_pt_2])
h_pm_bias, h_pm_mae, h_pm_rmse = dc.error_metrics(ec.loc[valid_pm_2, H_OBS].values, h_pm[valid_pm_2])
le_sw_bias, le_sw_mae, le_sw_rmse = dc.error_metrics(ec.loc[valid_sw_2, LE_OBS].values, le_sw[valid_sw_2])
le_pt_bias, le_pt_mae, le_pt_rmse = dc.error_metrics(ec.loc[valid_pt_2, LE_OBS].values,le_pt[valid_pt_2])
le_pm_bias, le_pm_mae, le_pm_rmse = dc.error_metrics(ec.loc[valid_pm_2, LE_OBS].values,le_pm[valid_pm_2])

h_sw_cor, h_sw_p_value, h_sw_slope, h_sw_intercept, h_sw_d = dc.agreement_metrics(ec.loc[valid_sw_2, H_OBS].values, 
                                                                                  h_sw[valid_sw_2])
h_pt_cor, h_pt_p_value, h_pt_slope, h_pt_intercept, h_pt_d = dc.agreement_metrics(ec.loc[valid_pt_2, H_OBS].values, 
                                                                                  h_pt[valid_pt_2])
h_pm_cor, h_pm_p_value, h_pm_slope, h_pm_intercept, h_pm_d = dc.agreement_metrics(ec.loc[valid_pm_2, H_OBS].values, 
                                                                                  h_pm[valid_pm_2])

le_sw_cor, le_sw_p_value, le_sw_slope, le_sw_intercept, le_sw_d = dc.agreement_metrics(ec.loc[valid_sw_2, LE_OBS].values, 
                                                                                       le_sw[valid_sw_2])
le_pt_cor, le_pt_p_value, le_pt_slope, le_pt_intercept, le_pt_d = dc.agreement_metrics(ec.loc[valid_pt_2, LE_OBS].values, 
                                                                                       le_pt[valid_pt_2])
le_pm_cor, le_pm_p_value, le_pm_slope, le_pm_intercept, le_pm_d = dc.agreement_metrics(ec.loc[valid_pm_2, LE_OBS].values, 
                                                                                       le_pm[valid_pm_2])

table_h = [["bias", h_sw_bias, h_pt_bias, h_pm_bias],
         ["RMSE", h_sw_rmse, h_pt_rmse, h_pm_rmse],
         ["MAE", h_sw_mae, h_pt_mae, h_pm_mae],
         ["Pearson", h_sw_cor, h_pt_cor, h_pm_cor],
         ["Willmot's d", h_sw_d, h_pt_d, h_pm_d]]
print("Error metrics for sensible heat flux")
print(tabulate(table_h, headers=["Metric", "TSEB-SW", "TSEB-PT", "TSEB-PM"]))

table_le = [["bias", le_sw_bias, le_pt_bias, le_pm_bias],
         ["RMSE", le_sw_rmse, le_pt_rmse, le_pm_rmse],
         ["MAE", le_sw_mae, le_pt_mae, le_pm_mae],
         ["Pearson", le_sw_cor, le_pt_cor, le_pm_cor],
         ["Willmot's d", le_sw_d, le_pt_d, le_pm_d]]

print("\n=======================================\n")
print("Error metrics for latent heat flux")
print(tabulate(table_le, headers=["Metric", "TSEB-SW", "TSEB-PT", "TSEB-PM"]))

## We save the results to ASCII tables

In [ ]:
# Define the output filename
out_dir = Path().absolute() / "output"
outfile = out_dir / f"FLX_US-{w_site.value}_FLUXNET2015_TSEB-PT-ROWS_HR.csv"

# Create the output folder in case it does not exist
if not out_dir.exists():
    out_dir.mkdir(parents=True)

# Convert the outputs to a Python dictionary
outdict = {"TIMESTAMP": ec["TIMESTAMP"].dt.strftime("%Y%m%d%H%M"),
           "FLAG_TSEBPT": flag_pt, "LE_TSEBPT": le_pt, "H_TSEBPT": h_pt, "NETRAD_TSEBPT": netrad_pt, "G_TSEBPT": g_pt, 
           "T_S_TSEBPT": ts_pt, "T_C_TSEBPT": tc_pt, "T_AC_TSEBPT": t_ac_pt, "LN_S_TSEBPT": ln_s_pt, "LN_C_TSEBPT": ln_c_pt, 
           "LE_C_TSEBPT": le_c_pt, "H_C_TSEBPT": h_c_pt, "LE_S_TSEBPT": le_s_pt, "H_S_TSEBPT": h_s_pt, "R_S_TSEBPT": r_s_pt, 
           "R_X_TSEBPT": r_x_pt, "R_A_TSEBPT": r_a_pt, "USTAR_TSEBPT": u_friction_pt, "L_MO_TSEBPT":lmo_pt, 
           "ITERATIONS_TSEBPT": n_iterations_pt}

# Crete the output dataframe and save it to csv
pd.DataFrame(outdict).to_csv(outfile, sep=";", na_rep=-9999, index=False)
print(f"Saved to {outfile}")

# Define the output filename
outfile = out_dir / f"FLX_US-{w_site.value}_FLUXNET2015_TSEB-SW-ROWS_HR.csv"

# Convert the outputs to a Python dictionary
outdict = {"TIMESTAMP": ec["TIMESTAMP"].dt.strftime("%Y%m%d%H%M"),
           "FLAG_TSEBSW": flag_sw, "LE_TSEBSW": le_sw, "H_TSEBSW": h_sw, "NETRAD_TSEBSW": netrad_sw, "G_TSEBSW": g_sw, 
           "T_S_TSEBSW": ts_sw, "T_C_TSEBSW": tc_sw, "T_AC_TSEBSW": t_ac_sw, "LN_S_TSEBSW": ln_s_sw, "LN_C_TSEBSW": ln_c_sw, 
           "LE_C_TSEBSW": le_c_sw, "H_C_TSEBSW": h_c_sw, "LE_S_TSEBSW": le_s_sw, "H_S_TSEBSW": h_s_sw, "R_S_TSEBSW": r_s_sw, 
           "R_X_TSEBSW": r_x_sw, "R_A_TSEBSW": r_a_sw, "USTAR_TSEBSW": u_friction_sw, "L_MO_TSEBSW":lmo_sw, 
           "ITERATIONS_TSEBSW": n_iterations_sw}

# Crete the output dataframe and save it to csv
pd.DataFrame(outdict).to_csv(outfile, sep=";", na_rep=-9999, index=False)
print(f"Saved to {outfile}")

# Define the output filename
outfile = out_dir / f"FLX_US-{w_site.value}_FLUXNET2015_TSEB-PM-ROWS_HR.csv"

# Convert the outputs to a Python dictionary
outdict = {"TIMESTAMP": ec["TIMESTAMP"].dt.strftime("%Y%m%d%H%M"),
           "FLAG_TSEBPM": flag_pm, "LE_TSEBPM": le_pm, "H_TSEBPM": h_pm, "NETRAD_TSEBPM": netrad_pm, "G_TSEBPM": g_pm, 
           "T_S_TSEBPM": ts_pm, "T_C_TSEBPM": tc_pm, "T_AC_TSEBPM": t_ac_pm, "LN_S_TSEBPM": ln_s_pm, "LN_C_TSEBPM": ln_c_pm, 
           "LE_C_TSEBPM": le_c_pm, "H_C_TSEBPM": h_c_pm, "LE_S_TSEBPM": le_s_pm, "H_S_TSEBPM": h_s_pm, "R_S_TSEBPM": r_s_pm, 
           "R_X_TSEBPM": r_x_pm, "R_A_TSEBPM": r_a_pm, "USTAR_TSEBPM": u_friction_pm, "L_MO_TSEBPM":lmo_pm, 
           "ITERATIONS_TSEBPM": n_iterations_pm}

# Crete the output dataframe and save it to csv
pd.DataFrame(outdict).to_csv(outfile, sep=";", na_rep=-9999, index=False)
print(f"Saved to {outfile}")

# Conclusions

* TSEB-SW seems to outperform TSEB-PT under advective conditions/sites with negative sensible heat fluxes at daytime
* However, TSEB-SW requires additional parametrization: minimuim stomata resistance and its relationship to the environmental conditions
* Both TSEB-PT and TSEB-SW seems to produce larger errors in Barelli 007, and Barelli 012* 
* [...]

:::{note}
Please feel free comment any thoughts. This is work in progress!!!
:::